# Cell Tracking and Lineage Construction in Live-Cell Imaging Data
---


### Global Imports

In [1]:
import os
import datetime
import errno
import argparse

import numpy as np

import deepcell

Using TensorFlow backend.


### Load the Training Data

In [2]:
# Training a tracking algorithm is a complicated process that requires alot of data
# We recommend combining multiple data sets  

# Download three different sets of data (saves to ~/.keras/datasets)
filename_3T3 = '3T3_NIH.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3.load_tracked_data(filename_3T3)
print('3T3 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HeLa = 'HeLa_S3.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hela_s3.load_tracked_data(filename_HeLa)
print('HeLa -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HEK = 'HEK293.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hek293.load_tracked_data(filename_HEK)
print('HEK293 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

3T3 -
X.shape: (192, 30, 154, 182, 1)
y.shape: (192, 30, 154, 182, 1)
HeLa -
X.shape: (144, 40, 216, 256, 1)
y.shape: (144, 40, 216, 256, 1)
HEK293 -
X.shape: (207, 30, 135, 160, 1)
y.shape: (207, 30, 135, 160, 1)


In [3]:
# Compile multiple TRK files into one
from deepcell.utils.tracking_utils import load_trks
from deepcell.utils.tracking_utils import save_trks

# Define a normalizaiton function for the raw images that can be run before padding
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

# Define all the trks to load
basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
trks_files = [os.path.join(basepath, filename_3T3), 
              os.path.join(basepath, filename_HeLa), 
              os.path.join(basepath, filename_HEK)]
#trks_files = [filename_3T3, filename_HeLa, filename_HEK]

# Each TRKS file may have differrent dimensions but the model expects uniform dimensions
# So we will determine max dimensions and zero pad as neccesary
max_frames = 1
max_y = 1
max_x = 1

for trks_file in trks_files:
    trks = load_trks(trks_file)

    # Store dimensions of raw and tracked to check new data against to pad if neccesary
    if trks['X'][0].shape[0] > max_frames:
        max_frames = trks['X'][0].shape[0]
    if trks['X'][0].shape[1] > max_y:
        max_y = trks['X'][0].shape[1]
    if trks['X'][0].shape[2] > max_x:
        max_x = trks['X'][0].shape[2]

# Load each trks file, normalize and pad as neccesary
lineages = []
X = []
y = []        

k = 0
movie_counter = 0
for trks_file in trks_files:
    trks = load_trks(trks_file)
    for i, (lineage, raw, tracked) in enumerate(zip(trks['lineages'], trks['X'], trks['y'])):
        movie_counter = k + i

        # Normalize the raw images
        for frame in range(raw.shape[0]):
            raw[frame, :, :, 0] = image_norm(raw[frame, :, :, 0]) 
            
        # Image padding if neccesary - This assumes that raw and tracked have the same shape
        if raw.shape[1] < max_y:
            diff2pad = max_y-raw.shape[1]
            pad_width = int(diff2pad/2)
            if diff2pad % 2 == 0:
                # Pad width can be split evenly
                raw = np.pad(raw, ((0,0), (pad_width,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (pad_width,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
            else:
                # Pad width cannot be split evenly
                raw = np.pad(raw, ((0,0), (pad_width+1,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (pad_width+1,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)

        if raw.shape[2] < max_x:
            diff2pad = max_x-raw.shape[2]
            pad_width = int(diff2pad/2)
            if diff2pad % 2 == 0:
                # Pad width can be split evenly
                raw = np.pad(raw, ((0,0), (0,0), (pad_width,pad_width), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (0,0), (pad_width,pad_width), (0,0)), mode='constant', constant_values=0)
            else:
                # Pad width cannot be split evenly
                raw = np.pad(raw, ((0,0), (0,0), (pad_width+1,pad_width), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (0,0), (pad_width+1,pad_width), (0,0)), mode='constant', constant_values=0)
        
        if raw.shape[0] < max_frames:   
            pad_width = int(max_frames-raw.shape[0])
            raw = np.pad(raw, ((0,pad_width), (0,0), (0,0), (0,0)), mode='constant', constant_values=0)
            tracked = np.pad(tracked, ((0,pad_width), (0,0), (0,0), (0,0)), mode='constant', constant_values=0)
        
        lineages.append(lineage)
        X.append(raw)
        y.append(tracked)
                
    k = movie_counter + 1

    
# Save the combined datasets into one trks file
filename = 'combined_data.trks'
save_trks(os.path.join(basepath, filename), lineages, X, y)

# View stats on this combined file
from deepcell.utils.tracking_utils import trks_stats
trks_stats(os.path.join(basepath, filename))

Image data shape:  (679, 40, 216, 256, 1)
Number of lineages (should equal batch size):  679
Dataset Statistics:
Total number of unique tracks (cells) -  10872
Total number of divisions             -  790
Average number of frames per track    -  25


### Set up filepath constants

In [4]:
# The path to the data file is currently required for `train_model_()` functions

# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

# DATA_FILE should be a trks file (contains 2 np arrays and a lineage dictionary)
DATA_FILE = os.path.join(DATA_DIR, filename)

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [5]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [6]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

tracking_model_name = 'tracking_model'

n_epoch = 10  # Number of training epochs
test_size = .10  # % of data saved as test

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Tracking training settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 5
neighborhood_scale_size=30
batch_size = 128  
crop_dim = 32

in_shape = (crop_dim, crop_dim, 1)
seed = 111 # Only needed for accuracy verification - if not desired set = None

### Training the Model

#### Instantiate the tracking model

In [7]:
from deepcell import model_zoo

tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

#### Option 1: Train a new tracking model

In [ ]:
from deepcell.training import train_model_siamese_daughter

tracking_model = train_model_siamese_daughter(
    model=tracking_model,
    dataset=DATA_FILE,  # full path to trks file
    model_name=tracking_model_name,
    optimizer=optimizer,
    batch_size=batch_size,
    crop_dim=crop_dim,
    min_track_length=min_track_length,
    features=features,
    neighborhood_scale_size=neighborhood_scale_size,
    n_epoch=n_epoch,
    model_dir=MODEL_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    class_weight=None,
    seed=seed)

training on dataset: /root/.keras/datasets/combined_data.trks
saving model at: /data/models/tracking_model.h5
saving loss at: /data/models/tracking_model.npz
X_train shape: (611, 40, 216, 256, 1)
y_train shape: (611, 40, 216, 256, 1)
X_test shape: (68, 40, 216, 256, 1)
y_test shape: (68, 40, 216, 256, 1)
Output Shape: (None, 3)
Training on 1 GPUs
Using real-time data augmentation.


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

total_train_pairs: 797200.0
total_test_pairs: 83024.0
batch size: 128
validation_steps:  648.0
Epoch 1/10
6227/6228 [============================>.] - ETA: 0s - loss: 0.0631 - acc: 0.9821
Epoch 00001: val_loss improved from inf to 0.11655, saving model to /data/models/tracking_model.h5
6228/6228 [==============================] - 2794s 449ms/step - loss: 0.0631 - acc: 0.9821 - val_loss: 0.1166 - val_acc: 0.9805
Epoch 2/10
6227/6228 [============================>.] - ETA: 0s - loss: 0.0306 - acc: 0.9941
Epoch 00002: val_loss did not improve from 0.11655
6228/6228 [==============================] - 2784s 447ms/step - loss: 0.0306 - acc: 0.9941 - val_loss: 0.1628 - val_acc: 0.9747
Epoch 3/10
6227/6228 [============================>.] - ETA: 0s - loss: 0.0243 - acc: 0.9961
Epoch 00003: val_loss improved from 0.11655 to 0.11406, saving model to /data/models/tracking_model.h5
6228/6228 [==============================] - 2757s 443ms/step - loss: 0.0243 - acc: 0.9961 - val_loss: 0.1141 - val_a

#### Option 2: Load an existing tracking model

In [7]:
# Re-instantiate the model and load weights
siamese_weights_file = 'tracking_model.h5'
siamese_weights_file = os.path.join(MODEL_DIR, PREFIX, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### (Optional) Investigate Model Performance with a Confusion Matrix - Requires a Seed Value

In [9]:
# Using DATA_FILE from above to extract Test Data 
# Change if you are not using `deepcell.datasets`

In [10]:
import deepcell.image_generators as generators
from deepcell.utils.data_utils import get_data

train_dict, test_dict = get_data(DATA_FILE, mode='siamese_daughters', seed=seed)

datagen_test = generators.SiameseDataGenerator(
        rotation_range=0,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=0,     # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=0, # randomly flip images
        vertical_flip=0)   # randomly flip images

test_iterator = generators.SiameseIterator(test_dict,
                                           datagen_test,
                                           neighborhood_scale_size=neighborhood_scale_size,
                                           min_track_length=min_track_length,
                                           features=features)

/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

In [11]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []
for i in range(1,1001):
    if i % 100 == 0:
        print(".", end="")
    lst, y_true = next(test_iterator)
    y_true = list(map(np.argmax, y_true))
    y_pred = list(map(np.argmax, tracking_model.predict(lst)))
    Y.extend(y_true)
    Y_pred.extend(y_pred)

print("")
cm = confusion_matrix(Y, Y_pred)
print(cm)

..........
[[10296    20    12]
 [   12 10486    55]
 [  172   435  9904]]


In [12]:
test_acc = sum(np.array(Y) == np.array(Y_pred)) / len(Y)
print('Accuracy across all three classes: ', test_acc)

# Normalize the diagonal entries of the confusion matrix
cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
# Diagonal entries are the accuracies of each class
print('Accuracy for each individual class [Different, Same, Daughter]: ', cm.diagonal())

Accuracy across all three classes:  0.9775101936799184
Accuracy for each individual class [Different, Same, Daughter]:  [0.99690163 0.99365109 0.94225098]


### Track Multiple Movies and Generate Track Files

#### Run the model

In [13]:
# Using DATA_FILE from above for example Test Data 
# Change if you are not using `deepcell.datasets`

In [33]:
# Normalize raw images if needed
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

for batch in range(test_dict['X'].shape[0]):
    for frame in range(test_dict['X'].shape[1]):
        test_dict['X'][batch, frame, :, :, 0] = image_norm(test_dict['X'][batch, frame, :, :, 0]) 

In [14]:
# The tracking model is used in concert with other processes to track cells
# Import the neccesary tracking functionality
import deepcell.tracking

# Define where cell tracks will be saved
TRACK_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'track_data', PREFIX))
TRACK_FILE_NAME = 'batch_'

# create directories if they do not exist
try:
    os.makedirs(TRACK_DIR)
except OSError as exc:  # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise

# Depending on the number of batches you may not want to track everything at once
num_batches = test_dict['X'].shape[0]

# Generate a cell track for each batch
for batch in range(num_batches):
    print('Batch: ', batch)
    trial = deepcell.tracking.cell_tracker(test_dict['X'][batch], test_dict['y'][batch],
                         tracking_model,
                         max_distance=50,
                         track_length=5, division=0.5, birth=0.9, death=0.9,
                         neighborhood_scale_size=30,
                         features=features)
    trial._track_cells()
    file_name = TRACK_FILE_NAME + str(batch).zfill(2) + '.trk'
    file_path = os.path.join(TRACK_DIR, file_name)
    trial.dump(file_path)

Batch:  0
Tracking frame 1


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

New track
Tracking frame 2
Tracking frame 3
Tracking frame 4
Tracking frame 5
Tracking frame 6
Tracking frame 7
New track
Tracking frame 8
Tracking frame 9
Tracking frame 10
Tracking frame 11
Tracking frame 12
Tracking frame 13
Tracking frame 14
Tracking frame 15
Tracking frame 16
Tracking frame 17
Tracking frame 18
Tracking frame 19
Tracking frame 20
Tracking frame 21
Tracking frame 22
Tracking frame 23
Tracking frame 24
Tracking frame 25
Tracking frame 26
Tracking frame 27
Tracking frame 28
Tracking frame 29
Tracking frame 30
Tracking frame 31
Tracking frame 32
Tracking frame 33
Tracking frame 34
Tracking frame 35
Tracking frame 36
Tracking frame 37
Tracking frame 38
Tracking frame 39
Batch:  1
Tracking frame 1
Tracking frame 2
Tracking frame 3
Tracking frame 4
Tracking frame 5
Tracking frame 6
Tracking frame 7
Tracking frame 8
Tracking frame 9
Tracking frame 10
Tracking frame 11
Tracking frame 12
Tracking frame 13
Tracking frame 14
Tracking frame 15
Tracking frame 16
Tracking frame 

#### Bundle individual track files (each batch) into one .trks file for review

In [15]:
from deepcell.utils.tracking_utils import trk_folder_to_trks
from deepcell.utils.tracking_utils import load_trks

# Define a name for the trks file
SET_NAME = 'all_batches.trks'

# Compile trk files into one trks file
trk_folder_to_trks(TRACK_DIR,SET_NAME)

### Review the results

In [16]:
# Load the file we created above to review
FILE_PATH = os.path.join(os.path.dirname(TRACK_DIR), SET_NAME)
trks = load_trks(FILE_PATH)

lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

#### Raw and Tracked Movies

In [33]:
# View tracked results of each batch as a video
# NB: This does not render well on GitHub
from IPython.display import HTML
from deepcell.utils.plot_utils import get_js_video

# Change this value to look at other batches of data
batch = 0

# Raw
HTML(get_js_video(raw, batch=batch, cmap='gray'))

In [34]:
# Tracked

# Scale the colors to match the max cell label
vmax = max(lineages[batch].keys())

HTML(get_js_video(tracked, batch=batch, cmap='cubehelix', vmin=0, vmax=vmax))

#### Save the Raw and Tracked Output as Images

In [37]:
import matplotlib.pyplot as plt

# Define where images (movies) will be saved
MOVIE_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'tracked_movies', PREFIX))

# create directories if they do not exist
try:
    os.makedirs(MOVIE_DIR)
except OSError as exc:  # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise

for batch in range(num_batches):
    print('Batch: ', batch)
    batch_folder = 'Batch_{:02}'.format(batch)
    BATCH_DIR = os.path.join(MOVIE_DIR, batch_folder)
    
    # create directories if they do not exist
    try:
        os.makedirs(BATCH_DIR)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

    # Scale the colors to match the max cell label
    vmax = max(lineages[batch].keys())

    channel = 0 # These images should only have one channel
    for i in range(raw.shape[1]):
        name_raw = os.path.join(BATCH_DIR,'raw_frame_{:02}_.png'.format(i))
        name_tracked = os.path.join(BATCH_DIR,'tracked_frame_{:02}_.png'.format(i))
        plt.imsave(name_raw, raw[batch, i, :, :, channel], cmap='gray')
        plt.imsave(name_tracked, tracked[batch,i, :, :, channel], cmap='cubehelix', vmin=0, vmax=vmax)

Batch:  0
Batch:  1
Batch:  2
Batch:  3
Batch:  4
Batch:  5
Batch:  6
Batch:  7
Batch:  8
Batch:  9


### Bechmarking

#### Save Cell Lineages in an ISBI-Formatted Output txt

The ISBI Cell Tracking Challenge requires a text file (man_track.txt) that represents a batch's cell lineage as an acyclic graph. The format of this file is as follows: Every line corresponds to a single track that is encoded by four numbers separated by a space -  
L B E P  
where L is a unique label of the track (label of markers, 16-bit positive value),  
B is a zero-based index of the frame in which the track begins,  
E is a zero-based index of the frame in which the track ends,  
P is the label of the parent track (0 is used when no parent is defined)

N.B. DeepCell's unique approach allows for cells to be tracked even if it momentarily leaves the frame. This is not possible in convential tracking algorithms, so ISBI considers a cell's track to have ended once it leaves the frame. We adjust the output here to keep with ISBI's formatting (ie. each track only contains contiguous frames).

In [38]:
# Load the file we created above and want to translate into ISBI format
FILE_PATH = os.path.join(os.path.dirname(TRACK_DIR), SET_NAME)
trks = load_trks(FILE_PATH)

lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

In [39]:
# Define where benchmark data will be saved
BENCHMARK_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'tracking_benchmarks', PREFIX))

# create directories if they do not exist
try:
    os.makedirs(BENCHMARK_DIR)
except OSError as exc:  # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise

In [40]:
# Adds a new track to the lineage and swap the labels accordingly in the images
def create_new_ISBI_track(batch_tracked, batch_info, old_label, frames, daughters, frame_div):
    
    new_track = len(batch_info.keys())
    new_label = new_track + 1
         
    batch_info[new_label] = {}
    batch_info[new_label]['old_label'] = old_label
    batch_info[new_label]['label'] = new_label

    batch_info[new_label]['frames'] = frames
    batch_info[new_label]['daughters'] = daughters
    batch_info[new_label]['frame_div'] = frame_div
    batch_info[new_label]['parent'] = None

    for frame in frames:
        batch_tracked[frame][batch_tracked[frame] == old_label] = new_label
        
    return batch_info, batch_tracked

In [41]:
# Check for contiguous tracks (tracks should only consist of consecutive tracks)
# Split one track into two if neccesary
def contig_tracks(label, batch_info, batch_tracked):
    
    original_label = label
    frames = batch_info[original_label]['frames']
    final_frame_idx = len(frames) - 1
       
    for frame_idx, frame in enumerate(frames):
        next_con_frame = frame + 1
        # If the next frame is available and contiguous we should move on to the next frame. Otherwise:
        # If the next frame is available and NONcontiguous we should separate this track into two 
        if frame_idx + 1 <= final_frame_idx and next_con_frame != frames[frame_idx + 1]:
            contig_end_idx = frame_idx

            next_trk_frames = frames[frame_idx+1:]
            daughters = batch_info[original_label]['daughters']
            frame_div = batch_info[original_label]['frame_div']
                                  
            # Create a new track to hold the information from this frame forward and add it to the batch
            batch_info, batch_tracked = create_new_ISBI_track(batch_tracked, batch_info, original_label, 
                                                                next_trk_frames, daughters, frame_div)
                        
            # Adjust the info for the current track to vacate the new track info
            batch_info[original_label]['frames'] = frames[0:contig_end_idx+1]
            batch_info[original_label]['daughters'] = []
            batch_info[original_label]['frame_div'] = None
                        
            # Because we are splitting tracks recursively, we stop here
            break
        
        # If the current frame is the last frame then were done
        # Either the last frame is contiguous and we don't alter batch_info
        # or it's not and it's been made into a new track by the previous iteration of the loop

    return batch_info, batch_tracked


In [42]:
# Translate track data to ISBI format and provide outputs for benchmarking
# Record lineage data in txt as it is generated

for batch, batch_info in enumerate(lineages):
    print('batch number: ', batch)
   
    # Build subdirectories to hold benchmark info
    B_SUB_DIR = os.path.join(BENCHMARK_DIR, '{:02}_RES'.format(batch+1))
    
    # Create directory if it doesn't exist
    try:
        os.makedirs(B_SUB_DIR)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise
    
    # Prepare output txt
    text_file = open(os.path.join(B_SUB_DIR, "res_track.txt"), "w")
    
    batch_tracked = tracked[batch]
    labels = list(batch_info.keys())
    max_label = max(labels)
    
    for label in labels:
        batch_info, batch_tracked = contig_tracks(label, batch_info, batch_tracked)
               
        first_frame = np.amin(batch_info[label]['frames'])          
        last_frame = np.amax(batch_info[label]['frames'])
        if batch_info[label]['parent']:
            parent = batch_info[label]['parent']
        else:
            parent = 0

        print(label, first_frame, last_frame, parent)
        text_file.write('{} {} {} {}\n'.format(label, first_frame, last_frame, parent))
        
        # Check if the track need to be split
        if max(batch_info.keys()) > max_label:
            # If so, a new track was added
            new_max_label = max(batch_info.keys())
            labels.append(new_max_label)
            max_label = new_max_label
        
    text_file.close()
    lineages[batch] = batch_info
    tracked[batch] = batch_tracked
        

batch number:  0
1 0 39 0
2 0 39 0
3 0 39 0
4 0 1 0
5 0 39 0
6 0 39 0
7 1 2 0
8 7 7 0
9 8 39 0
batch number:  1
1 0 39 0
2 0 32 0
3 0 39 0
4 0 1 0
5 0 39 0
6 0 39 0
7 0 39 0
8 0 39 0
9 33 39 2
10 33 35 2
11 36 39 10
batch number:  2
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 0 0
12 0 7 0
13 3 6 0
14 10 10 0
15 3 7 0
16 9 29 0
17 9 9 0
18 13 26 0
19 13 13 0
20 29 29 0
batch number:  3
1 0 29 0
2 0 13 0
3 0 20 0
4 0 13 0
5 0 29 0
6 0 29 0
7 0 6 0
8 0 3 0
9 0 20 0
10 0 29 0
11 0 29 0
12 0 29 0
13 0 29 0
14 0 29 0
15 0 13 0
16 0 23 0
17 0 29 0
18 0 29 0
19 3 29 0
20 4 29 8
21 4 4 8
22 5 29 21
23 7 29 7
24 7 29 7
25 8 29 0
26 14 29 2
27 14 18 2
28 21 29 9
29 21 29 9
30 24 28 16
31 24 29 16
32 17 20 0
33 16 25 0
34 25 29 0
35 22 29 0
36 29 29 0
batch number:  4
1 0 4 0
2 0 29 0
3 0 29 0
4 6 29 0
5 29 29 0
6 6 29 0
batch number:  5
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 10 0
12 0 29 0

#### Generate new images to match the new ISBI-formatted lineage data

In [45]:
from skimage.external.tifffile import imsave

for batch, batch_info in enumerate(lineages):
    print('batch number: ', batch)
    
    # check into appropriate benchmark subdirectory
    B_SUB_DIR = os.path.join(BENCHMARK_DIR, '{:02}_RES'.format(batch+1))
    
    channel = 0 # These images should only have one channel
    for i in range(raw.shape[1]):
        name_tracked = os.path.join(B_SUB_DIR,'mask{:03}.tif'.format(i))
        imsave(name_tracked, tracked[batch, i, :, :, channel].astype('uint16'))

batch number:  0
batch number:  1
batch number:  2
batch number:  3
batch number:  4
batch number:  5
batch number:  6
batch number:  7
batch number:  8
batch number:  9
